In [ ]:
# Install the package directly from GitHub
!pip install git+https://github.com/wcw100168/Cubed-Sphere-DG-Solver.git

# Advanced Physics: Shallow Water Equations

This tutorial runs a full physics simulation: **Williamson Case 2** (Global Steady State Zonal Flow).
We will use the **NumPy** backend for this demonstration.

In [ ]:
# Use NumPy backend for stability
import os

# Set JAX to CPU if imported indirectly, but we prefer NumPy for this demo
os.environ["JAX_PLATFORMS"] = "cpu"

import numpy as np
import matplotlib.pyplot as plt
from cubed_sphere.solvers import CubedSphereSWE, SWEConfig

> **Feature Highlight: Dynamic DG CFL Time-Stepping**
>
> We are proud to introduce our **Dynamic DG CFL Time-Stepping** engine. You no longer need to guess a stable `dt` or manually calculate the CFL condition. The solver automatically analyzes the grid structure, polynomial order ($N$), and the instantaneous flow field to compute the optimal stable time step for every iteration.


In [ ]:
# 1. Physics & Numerical Parameters
# Numerical Discretization
N = 16               # Polynomial Order. Higher N = better spectral accuracy, but requires smaller time steps.
R = 6.37122e6        # Earth's Radius (m). Scales the metric terms.
H_avg = 2998.0       # Standard Depth for Williamson Case 2 (Global Steady State Zonal Flow).
g = 9.80616          # Gravity (m/s^2).
target_cfl = 0.5     # Courant Number (safety factor).

# Configuration Dictionary
# We specify the physical parameters (R, g, H) and the solver handles the time-stepping instability.
config = SWEConfig(
    N=N, 
    R=R,             # Radius is critical for correct spherical geometry.
    backend='numpy', # Use 'jax' for accelerated execution on GPU/TPU.
    H_avg=H_avg,
    gravity=g,
    CFL=target_cfl 
    # 'dt' is handled by the Dynamic DG CFL engine.
)

solver = CubedSphereSWE(config)


In [ ]:
# Setup Williamson Case 2 Initial Condition
# Use the solver's built-in initialization to ensure consistent velocity field
state = solver.get_initial_condition(type="case2")

# Visualize Initial State Statistics
h_min = np.min(state[0])
h_max = np.max(state[0])
print(f"Initial Height range: [{h_min:.4e}, {h_max:.4e}]")
print(f"Initial State Shape: {state.shape}")

> **Note: Geometric Conservation**
>
> This simulation uses the **Exact (Analytical) Geometry** derived from the grid mapping. This ensures high-order accuracy. While the Strong Form formulation has excellent stability properties, minor mass conservation errors ($\sim 10^{-10}$) are expected over long integrations but are negligible for this short demonstration.


In [ ]:
# Run Simulation using Auto-Computed dt
t_span = (0.0, 5.0 * 24.0 * 3600.0) # 5 Days
print("Starting Integration...")
final_state = solver.solve(t_span, state) 
print("Integration Complete.")

In [ ]:
# Conservation Check
# Note: Since the output of solve is just the final state, we compare it to the initial state
initial_mass = np.sum(state[0])
final_mass = np.sum(final_state[0])
diff = abs(final_mass - initial_mass)
rel_error = diff / initial_mass

print(f"Initial Mass: {initial_mass:.5e}")
print(f"Final Mass:   {final_mass:.5e}")
print(f"Mass Error:   {diff:.5e}")
print(f"Rel Error:    {rel_error:.5e}")

# Check relative error (should be near machine precision for mass)
assert rel_error < 1e-8, f"Mass conservation failed! Relative error: {rel_error:.3e} exceeds tolerance 1e-8"

In [ ]:
# Plot Height Field
# For visualization, we access the internal geometry object directly.
# (Note: In production code, prefer using the public .vis module or callback system)
import numpy as np

faces = solver._impl.faces
# Ensure safe list conversion for dictionary keys
face_name = list(solver._impl.topology.FACE_MAP)[0] 
fg = faces[face_name]

# Unwrap Face 0 (P1)
# The state is stored as Conserved Variables: (h*sqrt_g, hu*sqrt_g, hv*sqrt_g)
# We divide by the metric term (sqrt_g) to recover physical height (h).
h_field = final_state[0, 0] / fg.sqrt_g 

plt.figure(figsize=(8, 6))

# Use contourf instead of imshow to correctly map data to non-uniform LGL nodes.
# This prevents artificial spatial distortion of the waves near the block boundaries.
plt.contourf(fg.alpha, fg.beta, h_field, levels=50, cmap='viridis')

plt.colorbar(label='Geopotential Height (m)')
plt.title(f"Block 0 ({face_name}) Final Height Field")
plt.xlabel(r'Local Coordinate $\alpha$ (radians)')
plt.ylabel(r'Local Coordinate $\beta$ (radians)')
plt.show()